In [1]:
import tensorflow as tf
print(tf.__version__)

2.0.0-rc0


In [1]:
class Account:
    def __init__(self, number, name):
        self.number = number
        self.name = name
        self.balance = 0
        
    def deposit(self, amount):  #存款動作: amount代表存入金額
        if amount <= 0:
            raise ValueError('must be positive')
        self.balance += amount
        
    def withdraw(self, amount): #取款動作: amount代表取款金額
        if amount <= self.balance:
            self.balance -= amount
        else:
            raise RuntimeError('balance not enough')

In [3]:
acct1 = Account('123–456–789', 'Justin') #開一個帳戶
acct1.deposit(100)
acct1.withdraw(30)
print(acct1.balance) #餘額是 70

70


8192

In [8]:
import numpy as np
import pandas as pd
import scipy.signal as signal

class data_stft:
    """ data train to stft. """
    
    def __init__(self, data_type):
        self.data_type = data_type
        self.ae_sample_rate = 5000000
        self.ae_window_size = 8192
        self.ae_overlap_size = self.ae_window_size*0.5
        
        self.vib_sample_rate = 10000
        self.curr_sample_rate = 10000
        self.vib_window_size = 8192
        self.vib_overlap_size = self.vib_window_size*0.5
        self.url = "./AE_data/ae/AEdate_10.lvm"
        
    def ae_stft(self, url, ae_sample_rate, ae_window_size):
        try:
            dt_loading = pd.read_fwf(url, widths=[10])[21:5000021] # samples : 4999168 (1024 * 4882)
        except:
            raise ValueError('data path error')
        ae_rawdata = np.array(dt_loading, dtype = np.float_).ravel()
        ae_rawdata = ae_rawdata - ae_rawdata.mean()
        #sample_rate = len(ae_rawdata)
        [f ,t , stft] = signal.spectral.spectrogram(ae_rawdata[:], ae_sample_rate, np.hamming(ae_window_size),\
                                                   nperseg=ae_window_size ,noverlap=ae_window_size*0.5, detrend=False,\
                                                   return_onesided=True, mode='magnitude')
        return f, t, stft
    
    def vib_stft(self, url, vib_sample_rate, vib_window_size):
        #""" the vibration of the spinder radial and axial axis. """"
        try:
            dt_loading = pd.read_fwf(url, widths=None, names=[])[23:-1] # loading all data
        except:
            raise ValueError('data path error')
        df = np.array(dt_loading[0].str.split()).tolist()  # segemntation five data Curr1, Curr2, Curr3, Sd_ra, Sd_ax
        df = np.array(df).ravel()
        df = df.reshape(-1,5).astype(float)                # reshape and transform type (strinf to float)
        #Curr_1 = df[:,0]
        #Curr_2 = df[:,1]
        #Curr_3 = df[:,2]
        Sd_ra = df[:,3]   # radial
        Sd_ax = df[:,4]   # axial
        #Curr_1 = Curr_1 - Curr_1.mean()
        #Curr_2 = Curr_2 - Curr_2.mean()
        #Curr_3 = Curr_3 - Curr_3.mean()
        Sd_ra = Sd_ra - Sd_ra.mean()
        Sd_ax = Sd_ax - Sd_ax.mean()
        # radial axis
        [f_rad,t_rad,stft_rad] = signal.spectral.spectrogram(Sd_ra, vib_sample_rate, np.hamming(vib_window_size),\
                                                             nperseg=vib_window_size,noverlap=vib_window_size*0.5, detrend=False,\
                                                             return_onesided=True, mode='magnitude')
        # axial axis
        [f_axi,t_axi,stft_axi] = signal.spectral.spectrogram(Sd_ax, vib_sample_rate, np.hamming(vib_window_size),\
                                                             nperseg=vib_window_size ,noverlap=vib_window_size*0.5, detrend=False,\
                                                             return_onesided=True, mode='magnitude')
        return f_rad, t_rad, stft_rad, f_axi, t_axi, stft_axi
    
    def equ_data(self, url, mask):
        #"""" the grinding process parameters of the grinder. """"
        try:
            gapminder = pd.read_csv(url, header=-1)
        except:
            raise ValueError('csv. path error')
        select_df = pd.DataFrame(gapminder)
        select_df.columns = ['Num','SetSpindle','SetWorkAxis','FreeAxis','Nan','SpindleLoad','RealSpindle','','RealWorkAxis',\
                            '','Zposition','Thickness','Time','mark']
        mask1 = select_df['Time'] >= mask
        df_mask = select_df[mask1]                             # the data are filtered out of not necessary part, keep of the AE data part
        
        T1 = select_df.loc[df_mask.index[0]:,'Thickness']      # select 'Thickness' columns from 367:
        T2 = select_df.loc[df_mask.index[1]:,'Thickness']
        T1.index = range(len(T1.index))                        # RangeIndex(start=0, stop=335, step=1)
        T2.index = range(len(T2.index))
        thickness_error = T1[:-1] - T2[:]                      # workpiece tinckness error
        thickness_error.loc[thickness_error<0] = 0
        
        Z_pos1 = select_df.loc[df_mask.index[0]:,'Zposition']  # select 'Z position' columns from 367:
        Z_pos2 = select_df.loc[df_mask.index[1]:,'Zposition']
        Z_pos1.index = range(len(Z_pos1.index))                # RangeIndex(start=0, stop=335, step=1)
        Z_pos2.index = range(len(Z_pos2.index))
        z_pos_error = Z_pos1[:-1] - Z_pos2[:]                  # z positon error
        z_pos_error.loc[z_pos_error<0] = 0
        
        s1_current1 = select_df.loc[df_mask.index[0]:,'SpindleLoad']
        s1_current1.index = range(len(s1_current1.index))

        return thickness_error, z_pos_error, s1_current1

In [23]:
data_stft1 = data_stft('AE')
url = "./AE_data/ae/AEdate_10.lvm"
[f ,t , stft] = data_stft1.ae_stft(url, 5000000, 8192)

In [37]:
data_stft2 = data_stft('equpment_parameters')
csv_file = "./AE_data/equ_para/20180104_104852 設備參數.csv"
mask = "2018/01/04 10:43:17"
[te, zpe, s1c] = data_stft2.equ_data(csv_file, mask)

In [2]:
import os
import numpy as np
from data_util import data_stft as ds

class data_train:
    """" experiment data path and excel. """
    def __init__(self, data_type):
        self.data_type = data_type
        self.path_ae = "./AE_data/"
        self.mask = "2018/01/04 10:43:17"
    
    def data_path(self, list_path):
        path_container = []
        classes = os.listdir(list_path)        # show all file of this path
        if classes[0].find("AE") >= 0 :  classes.sort(key = lambda x: int(x[7:-4]))      # AEdata_xx.lvm
        elif classes[0].find("Curr") >= 0 : classes.sort(key = lambda x: int(x[8:-4]))   # Current_xx.lvm     
        for class_ in classes:
            files_path = os.path.join(list_path, class_)  # series connection between the absolute path of
                                                          # the data and the classification of random select            
            path_container.append(files_path)
        return path_container
    
    def create_data_dict(self, ae_path, vib_path, equ_path, mask):
        data_dict = []
        ae_all_path = self.data_path(ae_path)
        vir_all_path = self.data_path(vib_path)
        equ_all_path = self.data_path(equ_path)
        url_equ = equ_all_path[0]
        [tikness_er, zpos_er, s1_curr] = ds('equpment_parameters').equ_data(url_equ, mask)
        for i in range(len(ae_all_path)):
            data_comb = np.hstack((ae_all_path[i], vir_all_path[i], tikness_er[i], zpos_er[i], s1_curr[i]))
            data_dict.append(data_comb)
        
        return data_dict
  

In [11]:
url_exp1_ae = "./AE_data/exp1/AE_data/"
url_exp1_vib = "./AE_data/exp1/Vib_data/"
url_exp1_equ = "./AE_data/exp1/Equ_data/"
url_exp2_ae = "./AE_data/exp2/AE_data/"
url_exp2_vib = "./AE_data/exp2/Vib_data/"
url_exp2_equ = "./AE_data/exp2/Equ_data/"
exp1_mask = "2018/01/04 10:43:17"
exp2_mask = "2018/01/04 14:44:56"
dt1 = data_train("AE")
# ae_p = dt1.data_path(url_exp1_equ)


ds1 = dt1.create_data_dict(url_exp1_ae, url_exp1_vib, url_exp1_equ, exp1_mask)
ds2 = dt1.create_data_dict(url_exp2_ae, url_exp2_vib, url_exp2_equ, exp1_mask)
print('exp1:', ds1)
print('exp2:', ds2)
ds3 = ds1+ds2
print('exp1 + exp2:', ds3)

exp1: [array(['./AE_data/exp1/AE_data/AEdate_5.lvm',
       './AE_data/exp1/Vib_data/Current_50.lvm', '0.0030000000000000027',
       '0.0008000000000000229', '37'], dtype='<U38'), array(['./AE_data/exp1/AE_data/AEdate_10.lvm',
       './AE_data/exp1/Vib_data/Current_51.lvm', '0.0',
       '0.0008000000000000229', '37'], dtype='<U38')]
exp2: [array(['./AE_data/exp2/AE_data/AEdate_11.lvm',
       './AE_data/exp2/Vib_data/Current_50.lvm', '0.0030000000000000027',
       '0.0008000000000000229', '37'], dtype='<U38')]
exp1 + exp2: [array(['./AE_data/exp1/AE_data/AEdate_5.lvm',
       './AE_data/exp1/Vib_data/Current_50.lvm', '0.0030000000000000027',
       '0.0008000000000000229', '37'], dtype='<U38'), array(['./AE_data/exp1/AE_data/AEdate_10.lvm',
       './AE_data/exp1/Vib_data/Current_51.lvm', '0.0',
       '0.0008000000000000229', '37'], dtype='<U38'), array(['./AE_data/exp2/AE_data/AEdate_11.lvm',
       './AE_data/exp2/Vib_data/Current_50.lvm', '0.0030000000000000027',
       '0.0008

In [7]:
# write to the text file
with open('data_series_test.txt', 'w', encoding = 'utf-8') as f:
    f.write(str(ds3))

In [311]:
# read from the text file
import numpy as np
ae_all_path = []
vir_all_path = []
tikness_er = []
zpos_er = []
s1_curr = []
with open('data_series_test.txt', 'r') as f:
    slist = f.read()
    print('read:', slist)
    curLine = slist.strip(" ").split("array(")
    for col in curLine[1:]:
        segment = col.strip(" ").split()
        ae_all_path.append(str(segment[0][2:-2]))
        vir_all_path.append(str(segment[1][1:-2]))
        tikness_er.append(float(segment[2][1:-2]))
        zpos_er.append(float(segment[3][1:-2]))
        s1_curr.append(float(segment[4][1:-3]))

print(ae_all_path)
print(vir_all_path)
print((tikness_er))
print(zpos_er)
print((s1_curr))

read: [array(['./AE_data/exp1/AE_data/AEdate_5.lvm',
       './AE_data/exp1/Vib_data/Current_50.lvm', '0.0030000000000000027',
       '0.0008000000000000229', '37'], dtype='<U38'), array(['./AE_data/exp1/AE_data/AEdate_10.lvm',
       './AE_data/exp1/Vib_data/Current_51.lvm', '0.0',
       '0.0008000000000000229', '37'], dtype='<U38'), array(['./AE_data/exp2/AE_data/AEdate_11.lvm',
       './AE_data/exp2/Vib_data/Current_50.lvm', '0.0030000000000000027',
       '0.0008000000000000229', '37'], dtype='<U38')]
['./AE_data/exp1/AE_data/AEdate_5.lvm', './AE_data/exp1/AE_data/AEdate_10.lvm', './AE_data/exp2/AE_data/AEdate_11.lvm']
['./AE_data/exp1/Vib_data/Current_50.lvm', './AE_data/exp1/Vib_data/Current_51.lvm', './AE_data/exp2/Vib_data/Current_50.lvm']
[0.0030000000000000027, 0.0, 0.0030000000000000027]
[0.0008000000000000229, 0.0008000000000000229, 0.0008000000000000229]
[37.0, 37.0, 37.0]


str

In [76]:
# get want using file path
from os import walk
from os.path import join

filepath = []
ae_path = []
cu_path = []
ae_path_container = []
cu_path_container = []
# 指定要列出所有檔案的目錄
mypath = "F:/AE訊號資料/20180104 第一次研磨(過負載)/"

# 遞迴列出所有檔案的絕對路徑
for root, dirs, files in walk(mypath):
    for f in files:
        if f.find("AEdate") >=0: ae_path.append(f)
        elif f.find("Current") >=0: cu_path.append(f)
        elif f.find(".csv") >=0: equ_path = f
    ae_path.sort(key = lambda x: int(x[7:-4]))
    cu_path.sort(key = lambda x: int(x[8:-4]))
    for class_ in ae_path:
        files_path = os.path.join(root, class_)           
        ae_path_container.append(files_path)
    for class_ in cu_path:
        files_path = os.path.join(root, class_)           
        cu_path_container.append(files_path)
    equ_path_container = equ_path
equ_path_container

'20180104_104852 設備參數.csv'

In [1]:
# using data_util.py
from data_util import data_train

dt1 = data_train("AE")
ds1 = dt1.create_data_dict("./AE_data/ae/","./AE_data/vib/","./AE_data/equ_para/")
print(ds1)

[array(['./AE_data/ae/exp1\\AEdate_10.lvm',
       './AE_data/vib/exp1\\Current_50.lvm', '0.0030000000000000027',
       '0.0008000000000000229', '37'], dtype='<U33'), array(['./AE_data/ae/exp2\\AEdate_11.lvm',
       './AE_data/vib/exp2\\Current_50.lvm', '0.0',
       '0.0008000000000000229', '37'], dtype='<U33'), array(['./AE_data/ae/exp3\\AEdate_12.lvm',
       './AE_data/vib/exp3\\Current_50.lvm', '0.0',
       '0.0007999999999999119', '37'], dtype='<U33')]


In [71]:
from data_util import data_stft

data_stft1 = data_stft('AE')
url = ds1[0][0]
[ae_f ,ae_t , ae_stft] = data_stft1.ae_stft(url, 5000000, 8192)
url2 = ds1[0][1]
[f_rad, t_rad, stft_rad, f_axi, t_axi, stft_axi] = data_stft1.vib_stft(url2, 10000, 1024) 

In [11]:
# Second part

In [1]:
import numpy as np
import pandas as pd
import scipy.signal as signal

class data_transform_stft:
    """ data train to stft. """
    
    def __init__(self, data_type):
        self.data_type = data_type
        self.ae_sample_rate = 5000000
        self.ae_window_size = 4096
        self.ae_overlap_size = self.ae_window_size*0.5
        
        self.vib_sample_rate = 10000
        self.curr_sample_rate = 10000
        self.vib_window_size = 256
        self.vib_overlap_size = self.vib_window_size // 2
        self.url = "/media/yukun/2C167E66167E30C6/AE_data/20180104 正常砂輪研磨/第一次研磨(過負載)/"
        self.csv_path = "/20180104_104852 設備參數.csv"
        self.mask = "2018/01/04 10:43:20"
        
    def ae_stft(self, url, ae_sample_rate, ae_window_size):
        try:
            dt_loading = pd.read_fwf(url, widths=[10])[21:5000021] # samples : 4999168 (1024 * 4882)
        except:
            raise ValueError('data path error')
        ae_rawdata = np.array(dt_loading, dtype = np.float_).ravel()
        ae_rawdata = ae_rawdata - ae_rawdata.mean()
        #sample_rate = len(ae_rawdata)        
        [f,t,Sxx]=signal.spectral.spectrogram(x = ae_rawdata[:],
                                              fs = ae_sample_rate,
                                              window = np.hamming(ae_window_size),
                                              nperseg = ae_window_size,
                                              noverlap = ae_window_size // 2,
                                              nfft = None,
                                              detrend = False,
                                              return_onesided = True,
                                              mode = 'magnitude')
        # get AE data start date
        dt_get_date = pd.read_fwf(url, widths=[20])[8:9]
        dt_datetype = np.array(dt_get_date, dtype = np.str_)[0]
        # get AE data start time
        dt_get_time = pd.read_fwf(url, widths=[13])[14:15] # samples : 4999168 (1024 * 4882) (2^22 = 4194304)
        dt_timetype = np.array(dt_get_time, dtype = np.str_)[0]
        # add date and time to be the mask
        self.mask = dt_datetype[0][5:]+' '+ dt_timetype[0][5:]
#         print('data mask date and time : {0}'.format(self.mask))

        return f, t, Sxx/np.max(Sxx)
    
    def vib_stft(self, url, vib_sample_rate, vib_window_size):
        #""" the vibration of the spinder radial and axial axis. """"
        try:
            dt_loading = pd.read_fwf(url, widths=None, names=[])[23:-1] # loading all data
        except:
            raise ValueError('data path error')
        df = np.array(dt_loading[0].str.split()).tolist()  # segemntation five data Curr1, Curr2, Curr3, Sd_ra, Sd_ax
        df = np.array(df).ravel()
        df = df.reshape(-1,5).astype(float)                # reshape and transform type (strinf to float)
        #Curr_1 = df[:,0]
        #Curr_2 = df[:,1]
        #Curr_3 = df[:,2]
        Sd_ra = df[:,3]   # radial
        Sd_ax = df[:,4]   # axial
        #Curr_1 = Curr_1 - Curr_1.mean()
        #Curr_2 = Curr_2 - Curr_2.mean()
        #Curr_3 = Curr_3 - Curr_3.mean()
        Sd_ra = Sd_ra - Sd_ra.mean()
        Sd_ax = Sd_ax - Sd_ax.mean()
        # radial axis
        [f_rad,t_rad,Sd_ra_stft]=signal.spectral.spectrogram(x = Sd_ra,
                                                             fs = vib_sample_rate,
                                                             window = np.hamming(vib_window_size),
                                                             nperseg = vib_window_size,
                                                             noverlap = vib_window_size // 2,
                                                             nfft = None,
                                                             detrend = False,
                                                             return_onesided = True,
                                                             mode = 'magnitude')
        # axial axis
        [f_axi,t_axi,Sd_ax_stft]=signal.spectral.spectrogram(x = Sd_ax,
                                                             fs = vib_sample_rate,
                                                             window = np.hamming(vib_window_size),
                                                             nperseg = vib_window_size,
                                                             noverlap = vib_window_size // 2,
                                                             nfft = None,
                                                             detrend = False,
                                                             return_onesided = True,
                                                             mode = 'magnitude')
        return f_rad, t_rad, Sd_ra_stft/np.max(Sd_ra_stft), f_axi, t_axi, Sd_ax_stft/np.max(Sd_ax_stft)
    
    def ae_vib_combined(self, url_ae, url_vib):
        [_ , _, ae_stft] = self.ae_stft(url_ae, self.ae_sample_rate, self.ae_window_size)
        [_ , _, Sd_ra_stft, _, _, Sd_ax_stft] = self.vib_stft(url_vib, self.vib_sample_rate, self.vib_window_size)
        
        return ae_stft, Sd_ra_stft, Sd_ax_stft
    
    def equ_data(self, url):
        #"""" the grinding process parameters of the grinder. """"
        try:
            gapminder = pd.read_csv(url, header=None)
        except:
            raise ValueError('csv. path error')
        select_df = pd.DataFrame(gapminder)
        select_df.columns = ['Num','SetSpindle','SetWorkAxis','FreeAxis','Nan','SpindleLoad',\
                             'RealSpindle','','RealWorkAxis','','Zposition','Thickness','Time','mark']
        time_mask = select_df['Time'] == self.mask
        present_index =  select_df[time_mask].index  # present data index
        previous_index = present_index - 1           # previous data index

        T1 = select_df.loc[select_df.index[present_index],'Thickness']
        T2 = select_df.loc[select_df.index[previous_index],'Thickness']
        thickness_error = np.array(T2) - np.array(T1)
        
        s1_current1 = select_df.loc[select_df.index[present_index],'SpindleLoad']

        return np.array(thickness_error), np.array(s1_current1)

In [23]:
# The data path
data_stft1 = data_transform_stft('AE and Vibration')
no = 29
data_path = "/media/yukun/2C167E66167E30C6/AE_data/20180104 正常砂輪研磨/第一次研磨(過負載)/"
url_ae = data_path + "AEdate_"+str(no)+".lvm"
url_vib = data_path + "Current_"+str(no)+".lvm"
url_csv = data_path + "20180104_104852 設備參數.csv"

In [25]:
# get the data stft
[_ , _, ae_stft] = data_stft1.ae_stft(url_ae, 5000000, 4096)
[_ , _, Sd_ra_stft, _, _, Sd_ax_stft] = data_stft1.vib_stft(url_vib, 10000, 256)
[error, current] = data_stft1.equ_data(url_csv)

In [26]:
print('ae stft shape (f, t): {}'.format(ae_stft.shape))
print('ra stft shape (f, t): {}'.format(Sd_ra_stft.shape))
print('ax stft shape (f, t): {}'.format(Sd_ax_stft.shape))
print('thickness error: {}'.format(error))
print('spindle current loading: {}'.format(current))

ae stft shape (f, t): (2049, 2440)
ra stft shape (f, t): (129, 77)
ax stft shape (f, t): (129, 77)
thickness error: [0.001]
spindle current loading: [37]


In [2]:
data_stft1 = data_transform_stft('AE and Vibration')
ae_train = []
ra_vib_train = []
ax_vib_train = []
thickness_train = []
for no in np.linspace(25,350,325):
    num = int(no)
    data_path = "/media/yukun/2C167E66167E30C6/AE_data/20180104 正常砂輪研磨/第一次研磨(過負載)/"
    url_ae = data_path + "AEdate_"+str(num)+".lvm"
    url_vib = data_path + "Current_"+str(num)+".lvm"
    url_csv = data_path + "20180104_104852 設備參數.csv"
    [_ , _, ae_stft] = data_stft1.ae_stft(url_ae, 5000000, 4096)
    [_ , _, Sd_ra_stft, _, _, Sd_ax_stft] = data_stft1.vib_stft(url_vib, 10000, 256)
    [error, _] = data_stft1.equ_data(url_csv)
    ae_train.append(ae_stft)
    ra_vib_train.append(Sd_ra_stft)
    ax_vib_train.append(Sd_ax_stft)
    thickness_train.append(error)

In [3]:
print('ae stft shape: {}'.format(np.array(ae_train).shape))
print('ra stft shape: {}'.format(np.array(ra_vib_train).shape))
print('ax stft shape: {}'.format(np.array(ax_vib_train).shape))
print('thickness shape: {}'.format(np.array(thickness_train).shape))

ae stft shape: (325, 2049, 2440)
ra stft shape: (325, 129, 77)
ax stft shape: (325, 129, 77)
thickness shape: (325, 1)


In [5]:
print(np.array(thickness_train))

[[0.   ]
 [0.003]
 [0.   ]
 [0.   ]
 [0.001]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.002]
 [0.   ]
 [0.   ]
 [0.001]
 [0.   ]
 [0.   ]
 [0.001]
 [0.   ]
 [0.   ]
 [0.001]
 [0.   ]
 [0.   ]
 [0.002]
 [0.   ]
 [0.   ]
 [0.001]
 [0.   ]
 [0.   ]
 [0.002]
 [0.   ]
 [0.   ]
 [0.002]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.002]
 [0.   ]
 [0.   ]
 [0.002]
 [0.   ]
 [0.   ]
 [0.002]
 [0.   ]
 [0.   ]
 [0.001]
 [0.   ]
 [0.   ]
 [0.002]
 [0.   ]
 [0.   ]
 [0.001]
 [0.   ]
 [0.   ]
 [0.002]
 [0.   ]
 [0.   ]
 [0.002]
 [0.   ]
 [0.   ]
 [0.002]
 [0.   ]
 [0.   ]
 [0.003]
 [0.   ]
 [0.   ]
 [0.002]
 [0.   ]
 [0.   ]
 [0.001]
 [0.   ]
 [0.   ]
 [0.002]
 [0.   ]
 [0.   ]
 [0.001]
 [0.   ]
 [0.   ]
 [0.003]
 [0.   ]
 [0.   ]
 [0.002]
 [0.   ]
 [0.   ]
 [0.002]
 [0.   ]
 [0.   ]
 [0.001]
 [0.   ]
 [0.   ]
 [0.002]
 [0.   ]
 [0.   ]
 [0.002]
 [0.   ]
 [0.   ]
 [0.002]
 [0.   ]
 [0.   ]
 [0.002]
 [0.   ]
 

In [ ]:
# data_path = "F:/AE訊號資料/20180104 第一次研磨(過負載)/"
# csv_file = data_path + "/20180104_104852 設備參數.csv"
# gapminder = pd.read_csv(csv_file, header=-1)  
# select_df = pd.DataFrame(gapminder)
# # print(select_df)
# # select_df.iloc[10]  # 讀取第 x row的所有資料
# select_df.columns = ['Num','SetSpindle','SetWorkAxis','FreeAxis','Nan','SpindleLoad','RealSpindle','','RealWorkAxis',\
#                      '','Zposition','Thickness','Time','mark']

# mask1 = select_df['Time'] >= "2018/01/04 10:43:17"
# df_mask = select_df[mask1] 
# df_mask

In [106]:
# time_mask = select_df['Time'] == "2018/01/04 10:43:51"  # 2018/01/04 10:43:17
# present_index =  select_df[time_mask].index  # present data index
# previous_index = present_index - 1           # previous data index
# print(present_index)
# print(previous_index)
# T1 = select_df.loc[select_df.index[present_index],'Thickness']
# T2 = select_df.loc[select_df.index[previous_index],'Thickness']
# print("T1: {}".format(np.array(T1)))
# print("T2: {}".format(np.array(T2)))
# thickness_error = np.array(T2) - np.array(T1)
# print("Thickness error: {}".format(thickness_error))
# s1_current = select_df.loc[select_df.index[present_index],'SpindleLoad']
# print("Spindle current: {}".format(np.array(s1_current)))

Int64Index([401], dtype='int64')
Int64Index([400], dtype='int64')
T1: [0.535]
T2: [0.536]
Thickness error: [0.001]
Spindle current: [37]


In [ ]:
import tensorflow as tf
tfrc_ae = tf.data.Dataset.from_tensor_slices(ae_train)
tfrc_ae

In [7]:
tfrc_vib_ra = tf.data.Dataset.from_tensor_slices(ra_vib_train)
tfrc_vib_ra

<TensorSliceDataset shapes: (129, 77), types: tf.float64>

In [8]:
tfrc_vib_ax = tf.data.Dataset.from_tensor_slices(ax_vib_train)
tfrc_vib_ax

<TensorSliceDataset shapes: (129, 77), types: tf.float64>

In [9]:
tfrc_thicknsss = tf.data.Dataset.from_tensor_slices(thickness_train)
tfrc_thicknsss

<TensorSliceDataset shapes: (1,), types: tf.float64>

In [28]:
for data1, data2, data3, data4 in zip(tfrc_ae, tfrc_vib_ra, tfrc_vib_ax, tfrc_thicknsss):
    print('ae : {}, vib_ra: {}, vib_ax: {}, thickness: {}'.format(data1, data2, data3, data4))

ae : [[1.74102199e-03 5.54870026e-04 1.50120012e-04 ... 1.29335970e-03
  2.35508967e-02 7.23100443e-03]
 [3.82633835e-04 1.31072064e-03 6.37810406e-04 ... 1.92158356e-02
  9.28379390e-03 5.78810024e-03]
 [1.47137131e-03 2.07411311e-03 9.21404195e-04 ... 2.00504112e-02
  1.69242146e-02 1.84452914e-03]
 ...
 [7.92521109e-05 2.56679652e-04 5.75285580e-05 ... 7.24780842e-04
  1.28566560e-04 2.16848898e-04]
 [8.71334211e-05 2.44467285e-04 1.09859374e-04 ... 5.60220811e-04
  3.65790502e-04 6.01285105e-05]
 [3.21048885e-04 2.82578535e-04 1.38615198e-04 ... 2.39431423e-04
  4.53030231e-04 1.74391062e-04]], vib_ra: [[0.01876542 0.01627198 0.01196524 ... 0.00907088 0.01042857 0.0161589 ]
 [0.00573095 0.00747936 0.00172899 ... 0.01940541 0.02415652 0.00221054]
 [0.00317948 0.01594747 0.01262812 ... 0.01027052 0.01633783 0.00955302]
 ...
 [0.00649852 0.01397967 0.02060207 ... 0.00862932 0.01860596 0.00800049]
 [0.01787517 0.01616716 0.01883112 ... 0.00597458 0.02944938 0.01144147]
 [0.01210774 0.0

ae : [[9.61423791e-02 4.86363542e-02 1.34480917e-01 ... 7.13416489e-03
  2.82622423e-03 1.06093011e-01]
 [5.07532157e-02 4.18232391e-02 1.37422243e-01 ... 4.58956279e-03
  2.40857953e-03 7.90387659e-02]
 [6.43320560e-02 1.46263076e-01 8.13186338e-02 ... 1.12382934e-03
  1.13107889e-03 3.52651032e-02]
 ...
 [7.97912557e-04 2.34832703e-03 1.51866921e-03 ... 3.12916595e-04
  1.54384924e-04 2.08913216e-03]
 [1.33519559e-03 4.26236473e-03 2.84945713e-03 ... 2.74743846e-04
  5.13389852e-05 1.35764740e-03]
 [8.98927965e-04 2.80446842e-03 4.56507592e-03 ... 1.93530721e-04
  1.51500060e-04 4.01811541e-04]], vib_ra: [[0.0093751  0.02035221 0.00229586 ... 0.01740975 0.00423114 0.00972788]
 [0.01193611 0.02020265 0.00769797 ... 0.0172352  0.0152987  0.02042308]
 [0.01166512 0.00272311 0.00747356 ... 0.01318267 0.00251056 0.02197303]
 ...
 [0.01404604 0.01685381 0.00789291 ... 0.0008545  0.00423664 0.02340787]
 [0.00410153 0.00158698 0.01343036 ... 0.01920244 0.02373255 0.02366352]
 [0.00061859 0.0

In [1]:
serialized_features_dataset = tf.data.Dataset.zip({"ae":tfrc_ae, "vib_ra":tfrc_vib_ra, "vib_ax":tfrc_vib_ax, "thickness":tfrc_thicknsss})
print(serialized_features_dataset)
for data in serialized_features_dataset.take(1):
    print('ae: {}, vib_ra: {}, vib_ax: {}, thickness: {}'.format(data['ae'], data['vib_ra'], data['vib_ax'], data['thickness']))

NameError: name 'tf' is not defined

In [27]:
# get want using file path
import os
from os import walk
from os.path import join

def data_path(mypath):
    filepath = []
    ae_path = []
    cu_path = []
    ae_path_container = []
    cu_path_container = []
    # 遞迴列出所有檔案的絕對路徑
    for root, dirs, files in walk(mypath):
        for f in files:
            if f.find("AEdate") >= 0: ae_path.append(f)
            elif f.find("Current") >= 0: cu_path.append(f)
            elif f.find(".csv") >=0: equ_path = f
        ae_path.sort(key = lambda x: int(x[7:-4]))
        cu_path.sort(key = lambda x: int(x[8:-4]))
        for class_ in ae_path:
            files_path = os.path.join(root, class_)           
            ae_path_container.append(files_path)
        for class_ in cu_path:
            files_path = os.path.join(root, class_)           
            cu_path_container.append(files_path)
        equ_path_container = equ_path
    return ae_path_container


mypath = "/media/yukun/2C167E66167E30C6/AE_data/20180104 正常砂輪研磨/第一次研磨(過負載)/"
x_data = data_path(mypath)
x_data

['/media/yukun/2C167E66167E30C6/AE_data/20180104 正常砂輪研磨/第一次研磨(過負載)/AEdate_25.lvm',
 '/media/yukun/2C167E66167E30C6/AE_data/20180104 正常砂輪研磨/第一次研磨(過負載)/AEdate_26.lvm',
 '/media/yukun/2C167E66167E30C6/AE_data/20180104 正常砂輪研磨/第一次研磨(過負載)/AEdate_27.lvm',
 '/media/yukun/2C167E66167E30C6/AE_data/20180104 正常砂輪研磨/第一次研磨(過負載)/AEdate_28.lvm',
 '/media/yukun/2C167E66167E30C6/AE_data/20180104 正常砂輪研磨/第一次研磨(過負載)/AEdate_29.lvm',
 '/media/yukun/2C167E66167E30C6/AE_data/20180104 正常砂輪研磨/第一次研磨(過負載)/AEdate_30.lvm',
 '/media/yukun/2C167E66167E30C6/AE_data/20180104 正常砂輪研磨/第一次研磨(過負載)/AEdate_31.lvm',
 '/media/yukun/2C167E66167E30C6/AE_data/20180104 正常砂輪研磨/第一次研磨(過負載)/AEdate_32.lvm',
 '/media/yukun/2C167E66167E30C6/AE_data/20180104 正常砂輪研磨/第一次研磨(過負載)/AEdate_33.lvm',
 '/media/yukun/2C167E66167E30C6/AE_data/20180104 正常砂輪研磨/第一次研磨(過負載)/AEdate_34.lvm',
 '/media/yukun/2C167E66167E30C6/AE_data/20180104 正常砂輪研磨/第一次研磨(過負載)/AEdate_35.lvm',
 '/media/yukun/2C167E66167E30C6/AE_data/20180104 正常砂輪研磨/第一次研磨(過負載)/AEdate_36.lvm',
 '/m

In [49]:
import tensorflow as tf
import pandas as pd
import numpy as np
tfrc_ae_filespath = tf.data.Dataset.from_tensor_slices(x_data)

for filepath in tfrc_ae_filespath.take(-1):
    list_ = filepath.numpy()
    print(list_)


b'F:/AE_signal_data/20180104_first_grinding(over_loading)/AEdate_25.lvm'
b'F:/AE_signal_data/20180104_first_grinding(over_loading)/AEdate_26.lvm'
b'F:/AE_signal_data/20180104_first_grinding(over_loading)/AEdate_27.lvm'
b'F:/AE_signal_data/20180104_first_grinding(over_loading)/AEdate_28.lvm'
b'F:/AE_signal_data/20180104_first_grinding(over_loading)/AEdate_29.lvm'
b'F:/AE_signal_data/20180104_first_grinding(over_loading)/AEdate_30.lvm'
b'F:/AE_signal_data/20180104_first_grinding(over_loading)/AEdate_31.lvm'
b'F:/AE_signal_data/20180104_first_grinding(over_loading)/AEdate_32.lvm'
b'F:/AE_signal_data/20180104_first_grinding(over_loading)/AEdate_33.lvm'
b'F:/AE_signal_data/20180104_first_grinding(over_loading)/AEdate_34.lvm'
b'F:/AE_signal_data/20180104_first_grinding(over_loading)/AEdate_35.lvm'
b'F:/AE_signal_data/20180104_first_grinding(over_loading)/AEdate_36.lvm'
b'F:/AE_signal_data/20180104_first_grinding(over_loading)/AEdate_37.lvm'
b'F:/AE_signal_data/20180104_first_grinding(over_lo

In [51]:
def parse_fn(dataset_path):
    window_size = 4096 
    dt_loading = pd.read_fwf(str(dataset_path)[2:-1], widths=[10])[21:5000021]
    ae_rawdata = np.array(dt_loading, dtype = np.float_).ravel()
    ae_rawdata = ae_rawdata - ae_rawdata.mean()       
    ae_ = ae_rawdata.astype(np.float32)
    stft = tf.signal.stft(
         ae_,
         window_size,
         frame_step=window_size//2,
         fft_length=None,
         window_fn=tf.signal.hann_window,
         pad_end=False,
         name=None
         )    
    return tf.abs(stft)

AUTOTUNE = tf.data.experimental.AUTOTUNE
batch_size = 5

train_data = tfrc_ae_filespath.batch(10)
train_data

<BatchDataset shapes: (None,), types: tf.string>

In [43]:
def ae_stft(url, ae_sample_rate, window_size):
    try:
        dt_loading = pd.read_fwf(url, widths=[10])[21:5000021] # samples : 4999168 (1024 * 4882)
    except:
        raise ValueError('data path error')
    ae_rawdata = np.array(dt_loading, dtype = np.float_).ravel()
    ae_rawdata = ae_rawdata - ae_rawdata.mean()       
    ae_ = ae_rawdata.astype(np.float32)
    stft = tf.signal.stft(
         ae_,
         window_size,
         frame_step=window_size//2,
         fft_length=None,
         window_fn=tf.signal.hann_window,
         pad_end=False,
         name=None
         )
    
    return tf.abs(stft)

In [24]:
def data_path(mypath, batch):
    filepath = []
    ae_path = []
    cu_path = []
    ae_path_container = []
    cu_path_container = []
    while True:
        for root, dirs, files in walk(mypath):
            for f in files:
                if f.find("AEdate") >= 0: ae_path.append(f)
                elif f.find("Current") >= 0: cu_path.append(f)
                elif f.find(".csv") >=0: equ_path = f
            ae_path.sort(key = lambda x: int(x[7:-4]))
            cu_path.sort(key = lambda x: int(x[8:-4]))
            for class1 in ae_path:
                files_path = os.path.join(root, class1)           
                ae_path_container.append(files_path)                
                if len(ae_path_container) >= batch:
                    yield ae_path_container
                    ae_path_container = []
            for class2 in cu_path:
                files_path = os.path.join(root, class2)           
                cu_path_container.append(files_path)
            equ_path_container = equ_path
            

mypath = "F:/AE_signal_data/20180104_first_grinding(over_loading)/"
x_data = data_path(mypath, 10)
print("1:{}".format(next(x_data)))
print("2:{}".format(next(x_data)))
print("3:{}".format(next(x_data)))

1:['F:/AE_signal_data/20180104_first_grinding(over_loading)/AEdate_25.lvm', 'F:/AE_signal_data/20180104_first_grinding(over_loading)/AEdate_26.lvm', 'F:/AE_signal_data/20180104_first_grinding(over_loading)/AEdate_27.lvm', 'F:/AE_signal_data/20180104_first_grinding(over_loading)/AEdate_28.lvm', 'F:/AE_signal_data/20180104_first_grinding(over_loading)/AEdate_29.lvm', 'F:/AE_signal_data/20180104_first_grinding(over_loading)/AEdate_30.lvm', 'F:/AE_signal_data/20180104_first_grinding(over_loading)/AEdate_31.lvm', 'F:/AE_signal_data/20180104_first_grinding(over_loading)/AEdate_32.lvm', 'F:/AE_signal_data/20180104_first_grinding(over_loading)/AEdate_33.lvm', 'F:/AE_signal_data/20180104_first_grinding(over_loading)/AEdate_34.lvm']
2:['F:/AE_signal_data/20180104_first_grinding(over_loading)/AEdate_35.lvm', 'F:/AE_signal_data/20180104_first_grinding(over_loading)/AEdate_36.lvm', 'F:/AE_signal_data/20180104_first_grinding(over_loading)/AEdate_37.lvm', 'F:/AE_signal_data/20180104_first_grinding(ov

NameError: name 'ae_path' is not defined